In [7]:
# Set project directory
import os
import sys


def project_dir():
    notebook_path = %pwd
    repo_name = "worldbank_data_exploration"
    repo_folder = notebook_path.split(repo_name)[0]
    return os.path.join(repo_folder, repo_name)


pwd = os.getenv("PWD", project_dir())
os.environ["PWD"] = pwd
sys.path.append(pwd)

In [8]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

from data_sources.get import get_indicators

In [9]:
df = get_indicators(
    [
        "SP.POP.GROW",
        "FP.CPI.TOTL.ZG",
        "SP.DYN.LE00.IN",
        "NE.EXP.GNFS.ZS",
        "NY.GDP.MKTP.KD.ZG",
        "SL.UEM.TOTL.ZS",
        "NV.AGR.TOTL.ZS",
        "EG.ELC.ACCS.ZS",
        "AG.LND.FRST.ZS",
        "SH.DYN.MORT",
        "NY.GDP.TOTL.RT.ZS",
        "SP.DYN.TFRT.IN",
        "EN.URB.LCTY.UR.ZS",
        "TG.VAL.TOTL.GD.ZS",
        "MS.MIL.XPND.GD.ZS",
    ]
)

print(df["Indicator Name"].unique())

['Population growth (annual %)' 'Inflation, consumer prices (annual %)'
 'Life expectancy at birth, total (years)'
 'Exports of goods and services (% of GDP)' 'GDP growth (annual %)'
 'Unemployment, total (% of total labor force) (modeled ILO estimate)'
 'Agriculture, forestry, and fishing, value added (% of GDP)'
 'Access to electricity (% of population)' 'Forest area (% of land area)'
 'Mortality rate, under-5 (per 1,000 live births)'
 'Total natural resources rents (% of GDP)'
 'Fertility rate, total (births per woman)'
 'Population in the largest city (% of urban population)'
 'Merchandise trade (% of GDP)' 'Military expenditure (% of GDP)']


In [10]:
df = df.pivot_table(
    values="Value", index="Year", columns=["Indicator Name", "Country Name"]
)

In [11]:
df_nans = df.loc[slice(2000, 2020)].isnull().sum().reset_index()

In [12]:
aggregates = np.array(
        ['Africa Eastern and Southern', 'Africa Western and Central', 'Arab World', 'Caribbean small states',
         'Central Europe and the Baltics', 'Early-demographic dividend', 'East Asia & Pacific',
         'East Asia & Pacific (excluding high income)', 'East Asia & Pacific (IDA & IBRD countries)', 'Euro area',
         'Europe & Central Asia', 'Europe & Central Asia (excluding high income)',
         'Europe & Central Asia (IDA & IBRD countries)', 'European Union', 'Fragile and conflict affected situations',
         'Heavily indebted poor countries (HIPC)', 'High income', 'IBRD only', 'IDA & IBRD total', 'IDA blend',
         'IDA only', 'IDA total', 'Late-demographic dividend', 'Latin America & Caribbean',
         'Latin America & Caribbean (excluding high income)', 'Latin America & the Caribbean (IDA & IBRD countries)',
         'Least developed countries: UN classification', 'Low & middle income', 'Low income', 'Lower middle income',
         'Middle East & North Africa', 'Middle East & North Africa (excluding high income)',
         'Middle East & North Africa (IDA & IBRD countries)', 'Middle income', 'North America', 'OECD members',
         'Other small states', 'Pacific island small states', 'Post-demographic dividend', 'Pre-demographic dividend',
         'Small states', 'South Asia', 'South Asia (IDA & IBRD)', 'Sub-Saharan Africa',
         'Sub-Saharan Africa (excluding high income)', 'Sub-Saharan Africa (IDA & IBRD countries)',
         'Upper middle income', 'World'])
aggregates.shape

(48,)

In [13]:
countries_with_nans = df_nans[df_nans[0] > 2]["Country Name"].unique()
countries_with_nans

array(['Afghanistan', 'Algeria', 'Cabo Verde', 'Congo, Rep.',
       'Equatorial Guinea', 'Georgia', 'Guinea-Bissau', 'Iraq',
       'Kiribati', "Korea, Dem. People's Rep.", 'Lebanon', 'Liberia',
       'Malaysia', 'Myanmar', 'Sierra Leone', 'Somalia', 'South Sudan',
       'Syrian Arab Republic', 'Turkey', 'Tuvalu', 'Armenia', 'Aruba',
       'Bahrain', 'Barbados', 'Bermuda', 'Cayman Islands',
       'Central African Republic', 'Djibouti', 'Eritrea',
       'French Polynesia', 'Greenland', 'Isle of Man', 'Kosovo', 'Libya',
       'Liechtenstein', 'Maldives', 'Nauru', 'New Caledonia',
       'San Marino', 'Sint Maarten (Dutch part)', 'Solomon Islands',
       'St. Lucia', 'Venezuela, RB', 'Curacao', 'Dominica', 'Ethiopia',
       'Guyana', 'Lao PDR', 'Lesotho', 'Northern Mariana Islands',
       'Papua New Guinea', 'Suriname', 'Virgin Islands (U.S.)',
       'Yemen, Rep.', 'Andorra', 'Marshall Islands', 'Palau',
       'St. Kitts and Nevis', 'Sudan', 'World', 'American Samoa',
       '

In [14]:
df_cleared = df.loc[2000:2020].stack()
print(df_cleared.shape)
df_cleared.drop(index=aggregates, level=1, inplace=True)
print(df_cleared.shape)
df_cleared.drop(index=countries_with_nans, level=1, inplace=True)
print(df_cleared.shape)

(5565, 15)
(4557, 15)
(2646, 15)


In [15]:
df_cleared.bfill(inplace=True)
df_cleared.ffill(inplace=True)

In [16]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df_cleared.values)
df_scaled = pd.DataFrame(
    data=data_scaled, index=df_cleared.index, columns=df_cleared.columns
)
df_scaled.head()

Indicator Name            Access to electricity (% of population)  \
Year Country Name                                                   
2000 Albania                                             0.532069   
     Angola                                              0.428605   
     Antigua and Barbuda                                 0.428605   
     Argentina                                           0.346849   
     Australia                                           0.532069   

Indicator Name            Agriculture, forestry, and fishing, value added (% of GDP)  \
Year Country Name                                                                      
2000 Albania                                                       1.642440            
     Angola                                                       -0.380091            
     Antigua and Barbuda                                          -0.816532            
     Argentina                                                    -0.485749            
     Australia                                                    -0.653662            

Indicator Name            Exports of goods and services (% of GDP)  \
Year Country Name                                                    
2000 Albania                                             -0.778195   
     Angola                                               1.493636   
     Antigua and Barbuda                                  1.784783   
     Argentina                                           -1.076300   
     Australia                                           -0.800961   

Indicator Name            Fertility rate, total (births per woman)  \
Year Country Name                                                    
2000 Albania                                             -0.324914   
     Angola                                               3.123411   
     Antigua and Barbuda                                 -0.291831   
     Argentina                                           -0.028706   
     Australia                                           -0.633432   

Indicator Name            Forest area (% of land area)  GDP growth (annual %)  \
Year Country Name                                                               
2000 Albania                                 -0.206285               0.853937   
     Angola                                   1.425968              -0.023033   
     Antigua and Barbuda                     -0.520747               0.686550   
     Argentina                               -0.962979              -0.889192   
     Australia                               -0.726554               0.170273   

Indicator Name            Inflation, consumer prices (annual %)  \
Year Country Name                                                 
2000 Albania                                          -0.488724   
     Angola                                           32.087535   
     Antigua and Barbuda                              -0.416367   
     Argentina                                        -0.046876   
     Australia                                        -0.046876   

Indicator Name            Life expectancy at birth, total (years)  \
Year Country Name                                                   
2000 Albania                                             0.191766   
     Angola                                             -3.293341   
     Antigua and Barbuda                                 0.189860   
     Argentina                                           0.143617   
     Australia                                           0.862432   

Indicator Name            Merchandise trade (% of GDP)  \
Year Country Name                                        
2000 Albania                                 -0.672987   
     Angola                                   1.223276   
     Antigua and Barbuda                     -0.283113   
     Argentina                               -1.156961   
     Australia                               -0